We plan to explore the relationship between COVID-19 death rates and vaccination rates across US states by visually examining their correlation. This analysis will involve gathering COVID-19 related data from the CDC's API and then extensively processing it to merge the various datasets. Since the population sizes of states vary significantly, we will focus on comparing rates rather than absolute numbers. To facilitate this, we will also source population data from the US Census to accurately calculate these rates.

1.  Get an API key from the US Census at <https://api.census.gov/data/key_signup.html>. Save it an object called `census_key`.


In [ ]:
census_key <- "91e9a6486ee655ba5b59cf4a2aac4169d13ae7a5"

2.  Use an internet search to find information about the US Census APIs to obtain vintage population estimates. Define a variable called `api` with the URL.


In [ ]:
api <- "https://api.census.gov/data/2021/pep/population"

3.  Read the documentation for the API and use the **httr2** package to prepare a request that gives us 2020 and 2021 population estimates for each state. Save the request in a variable called `request`, without performing it. Compare the request to the examples in the documentation to see if you are on the right track.


In [ ]:
library(httr2)
request <- request(api) |> 
    req_url_query(get = I("POP_2020,POP_2021,NAME"), 
                'for' = I("state:*"), 
                key = census_key)
request

4.  Now perform the request and examine the data that is returned. Save the result to `request` (overwrite it).


In [ ]:
request <- request |> req_perform()
request

5.  We see the request returned data in JSON format. We can see the content with the function `req_body_json`, but we want a data frame. Use the **jsonlite** package to convert the raw JSON data into a data frame. Save it in `population`.


In [ ]:
library(jsonlite)
population <- request |> resp_body_string() |>
    fromJSON(flatten = TRUE)
population

6.  Examine the table. Notice 1) it is not tidy, 2) the column types are not what we want, and 3) the first row is a header. Convert `population` to a tidy dataset. Remove the state ID column and change the name of the column with state names to `state_name`. Add a column with state abbreviations called `state`. Make sure you assign the abbreviations for DC and PR correctly. Hint: Start by using the **janitor** package to make the first row the header.


In [ ]:
library(tidyverse)
library(janitor)
population <- population |> row_to_names(1) |>
    as_tibble() |>
    select(-state) |>
    rename(state_name = NAME) |>
    pivot_longer(-state_name, names_to ="year", values_to = "population") |>
    mutate(year = str_remove(year, "POP_")) |>
    mutate(across(-state_name, as.numeric)) |>
    mutate(state = state.abb[match(state_name, state.name)]) |>
    mutate(state = case_when(
        state_name == "District of Columbia" ~ "DC",
        state_name == "Puerto Rico" ~ "PR",
        .default = state))
population

7.  As a check, make a barplot of states' 2021 populations


In [ ]:
population %>%
  filter(year == "2021") %>%
  ggplot(aes(x = state, y = population/10^6)) +
  geom_bar(stat = "identity", fill = "skyblue", color = "black") +
  labs(title = "States 2021 Populations", x = "States", y = "Populations in 2021") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 8))

8.  To practice `pivot_wider` make a bar plot of percent change in population by state.


In [ ]:
library(tidyverse)
library(tidyr)
library(ggplot2)
# Calculate percent change
pop_change <- population |>
  group_by(state_name) |>
  mutate(percent_change = ((population - lag(population)) / lag(population)) * 100)

# Percent change
pop_wide <- pop_change %>%
  pivot_wider(names_from = year, values_from = percent_change)

# Create a bar plot of percent change for 2021
ggplot(pop_change, aes(x = state_name, y = percent_change, fill = state_name)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Percent Change in Population by State (2020-2021)",
       x = "State",
       y = "Percent Change") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

9.  Add the following region numbers to the data:


In [ ]:
cdc_regions_list <- list(
  "1" = c("Connecticut", "Maine", "Massachusetts", "New Hampshire", "Rhode Island", "Vermont"),
  "2" = c("New Jersey", "New York", "Puerto Rico", "Virgin Islands"),
  "3" = c("Delaware", "District of Columbia", "Maryland", "Pennsylvania", "Virginia", "West Virginia"),
  "4" = c("Alabama", "Florida", "Georgia", "Kentucky", "Mississippi", "North Carolina", "South Carolina", "Tennessee"),
  "5" = c("Illinois", "Indiana", "Michigan", "Minnesota", "Ohio", "Wisconsin"),
  "6" = c("Arkansas", "Louisiana", "New Mexico", "Oklahoma", "Texas"),
  "7" = c("Iowa", "Kansas", "Missouri", "Nebraska"),
  "8" = c("Colorado", "Montana", "North Dakota", "South Dakota", "Utah", "Wyoming"),
  "9" = c("Arizona", "California", "Hawaii", "Nevada", "American Samoa", "Commonwealth of the Northern Mariana Islands", "Federated States of Micronesia", "Guam", "Marshall Islands", "Republic of Palau"),
  "10" = c("Alaska", "Idaho", "Oregon", "Washington"))

cdc_regions <- do.call(rbind, lapply(names(cdc_regions_list), function(region) {
  data.frame(region = region, state_name = cdc_regions_list[[region]])
})) |>
  mutate(region = factor(as.numeric(region)))

In [ ]:
population <- population %>%
  left_join(cdc_regions, by = "state_name")
  population

10. Go to <https://data.cdc.gov/> and learn about the CDC API for COVID-19 data. Find an API that provides state level data from SARS-COV2 cases and store it in a data frame.


In [ ]:
library(jsonlite)
api_cdc <- "https://data.cdc.gov/resource/pwn4-m3yp.json"
cases <- request(api_cdc) |> 
  req_url_query("$limit" = 1000000000) |>
  req_perform() |> resp_body_string() |> 
  fromJSON(flatten = TRUE)

11. Note that we obtained weekly data. Wrangle the table so that keep only states for which you have population data. Keep the following variables: state, epidemiological week and year, and new cases (as numbers). Order by state, then chronologically. Hint: Use `as_date`, `epiweek` and `epiyear` functions in **lubridate** package.


In [ ]:
library(lubridate)
cases <- cases |> as_tibble()
cases <- mutate(cases, date_updated = as.Date(date_updated))

cases <- cases |>
  mutate(
    iso_week = isoweek(date_updated),
    iso_year = isoyear(date_updated)
  ) |>
  rename(epi_week = iso_week, epi_year = iso_year)

cases_filtered <- cases |> 
  mutate(state_name = state.name[match(state, state.abb)]) |>
  select(state, epi_week, epi_year, new_cases) |>
  arrange(state, cases$date_updated) |>
  filter(state != "NYC")

In [ ]:
cases_filtered

12. Now repeat the same exercise for hospitalizations. However, before you start, notice the code for extracting the data is the same as in the previous exercise except for the API URL. Write a function that takes an API URL and returns the data in a data frame.


In [ ]:
hosp_api <- "https://data.cdc.gov/resource/39z2-9zu6.json"

13. Now use the function to get the raw hospital data. Examine the data once you have it to determine if its daily or weekly.


In [ ]:
library(jsonlite)
hosp_raw <- request(hosp_api) |> 
  req_url_query("$limit" = 1000000000) |>
  req_perform() |> resp_body_string() |> 
  fromJSON(flatten = TRUE)

hosp_raw <- hosp_raw |> as_tibble()

14. Collapse the data into weekly data and keep the same columns as in the cases dataset, except keep total weekly hospitalizations instead of cases. Remove weeks with less than 7 days reporting.


In [ ]:
library(tidyverse)
hosp_raw <- hosp_raw |>
  mutate(collection_date = as.Date(collection_date),
         epi_year = isoyear(collection_date),
         epi_week = isoweek(collection_date))

#remove weeks without reporting (7 day average)
hosp_data <- hosp_raw  |>
  filter(total_hospitalized_covid_1 >=1) |>
  rename(state = jurisdiction) |>
  filter(!grepl("Region", state, ignore.case = TRUE)) |>
  filter(!grepl("USA", state, ignore.case = TRUE))

#filter
hosp_filtered <- hosp_data |>
  mutate(state_name = state.name[match(state, state.abb)]) |>
  mutate(total_hospitalized_covid_1=as.numeric(total_hospitalized_covid_1), rm.na = TRUE) |>
  select(state, epi_week, epi_year, total_hospitalized_covid_1)
hosp_filtered

15. Repeat what you did in the previous two exercises for provisional COVID-19 deaths.


In [ ]:
death_api <- "https://data.cdc.gov/resource/r8kw-7aab.json"

deaths_raw <- request(death_api) |> 
  req_url_query("$limit" = 1000000000) |>
  req_perform() |> resp_body_string() |> 
  fromJSON(flatten = TRUE)

deaths_raw <- deaths_raw |> as_tibble()

deaths_raw <- deaths_raw |>
  mutate(week_ending_date = as.Date(week_ending_date),
         epi_year = year,
         epi_week = mmwr_week)

death_filtered <- deaths_raw |>
  rename(state_name = state) |>
  mutate(epi_year = str_remove(epi_year, "2019/")) |>
  filter(!grepl("United States", state_name, ignore.case = TRUE)) |>
  mutate(epi_week=as.numeric(epi_week), rm.na = TRUE) |>
  mutate(epi_year=as.numeric(epi_year), rm.na = TRUE) |>
  mutate(state = state.abb[match(state_name, state.name)]) |>
  select(state, state_name, epi_week, epi_year, covid_19_deaths) |>
  filter(!is.na(covid_19_deaths) & !is.na(epi_week))

16. Obtain vaccination data. Keep the variables `series_complete` and `booster` along with state and date. Remember we will later want to join with the others.


In [ ]:
vax_api <- "https://data.cdc.gov/resource/rh2h-3yt2.json"

vax_raw <- request(vax_api) |> 
  req_url_query("$limit" = 1000000000) |>
  req_perform() |> resp_body_string() |> 
  fromJSON(flatten = TRUE)

vax_raw <- vax_raw |> as_tibble()

vax_raw <- vax_raw |>
  mutate(date = as.Date(date),
  epi_year = isoyear(date),
  epi_week = mmwr_week)

vax_filtered <- vax_raw |>
  rename(state = location) |>
  mutate(state_name = state.name[match(state, state.abb)]) |>
  mutate(epi_week=as.numeric(epi_week), rm.na = TRUE) |>
  mutate(state_name = case_when(
    state == "DC" ~ "District of Columbia",
    state ==  "PR" ~ "Puerto Rico",
    .default = state_name)) |>
  filter(!grepl("VI", state, ignore.case = TRUE)) |>
  select(state, state_name, date, contains("booster"), contains("series_complete"), epi_year, epi_week)

In [ ]:
vax_filtered

17. Now we are ready to join the tables. We will only consider 2020 and 2021 as we don't have population sizes for 2020. However, because we want to guarantee that all dates are included we will create a data frame with all possible weeks. We can use this:


In [ ]:
all_dates <- data.frame(date = seq(make_date(2020, 1, 25), make_date(2021, 12, 31), by = "week")) |>
  mutate(date = ceiling_date(date, unit = "week", week_start = 7) - days(1)) |>
  mutate(mmwr_year = epiyear(date), mmwr_week = epiweek(date)) 

dates_and_pop <- cross_join(population, all_dates) |> rename(epi_week = mmwr_week, epi_year = mmwr_year) 

Use this to join all the tables in a way that preserves these dates. Call the final table `dat`.


In [ ]:
library(tidyverse)
dat <- dates_and_pop|> 
  left_join(cases_filtered, by = c("state", "epi_week", "epi_year")) |>
  left_join(hosp_filtered, by = c("state", "epi_week", "epi_year")) |>
  left_join(death_filtered, by = c("state", "epi_week", "epi_year", "state_name")) |>
  left_join(vax_filtered, by = c("state", "epi_week", "epi_year", "state_name"))

dat <- dat |> mutate(new_cases=as.numeric(new_cases), rm.na = TRUE)

In [ ]:
dat <- dat |> mutate(covid_19_deaths=as.numeric(covid_19_deaths), rm.na = TRUE)
dat <- dat |> mutate(series_complete_daily=as.numeric(series_complete_daily), rm.na = TRUE)
dat <- dat |> mutate(booster_daily=as.numeric(booster_daily), rm.na = TRUE)

18. Plot a trend plot with cases per person for all states with color representing region.


In [ ]:
dat |>
ggplot(aes(date.x, new_cases/population *100000, color = region)) +
geom_point() +
geom_line() +
labs(title = "Trend Plot of Cases per Person by State",
       x = "Date",
       y = "Cases per Person")

19. Repeat the above for hospitalizations and deaths. Use `pivot_longer` and `facet_wrap`. Plot rates per $100,000$ people. Place the plots on top of each other.


In [ ]:
names(dat)

In [ ]:
dat_long <- dat |>
  pivot_longer(cols = c(new_cases, total_hospitalized_covid_1, covid_19_deaths), names_to = "variable", values_to = "count") |>
  mutate(rate_per_100k = count / population * 100000)

# Plotting with ggplot2
ggplot(dat_long, aes(date.x, rate_per_100k, color = region)) +
  geom_point() +
  geom_line() +
  labs(title = "Trend Plot of Rates per 100,000 People by State",
       x = "Date",
       y = "Rate per 100,000 People") +
  facet_wrap(~variable, scales = "free_y", ncol = 1)

20. For the period January 1 to July 1, 2021 compute the deaths per day per 100,000 people in each state as well as the vaccination rate (primary series) by September 1st. Plot these against each other.


In [ ]:
dat_20 <- dat %>%
  filter(date.x >= as.Date("2021-01-01") & date.x <= as.Date("2021-07-01")) |>
  group_by(state, region) |>
  summarise(death_rate = mean(covid_19_deaths, na.rm = TRUE)*n() / population[1]*100000, 
  vax_rate = max(series_complete_daily)/population[1]*100)
  

dat_20 |> ggplot(aes(x = death_rate, y = vax_rate, color = region)) + 
geom_point() + 
labs(title = "Deaths per Day per 100,000 vs. Vaccination Rate by State", 
x = "Deaths per Day per 100,000", 
y = "Vaccination Rate", color = "state") + 
theme_minimal()

21. Repeat the exercise for the booster for October 1 to December 31, 2021.


In [ ]:
dat_21 <- dat %>%
  filter(date.x >= as.Date("2021-10-01") & date.x <= as.Date("2021-12-31")) |>
  group_by(state, region) |>
  summarise(death_rate = mean(covid_19_deaths, na.rm = TRUE)*n() / population[1]*100000, 
  booster_rate = max(booster_daily)/population[1]*100)
  

dat_21 |> ggplot(aes(x = death_rate, y = booster_rate, color = region)) + 
geom_point() + 
labs(title = "Deaths per Day per 100,000 vs. Vaccination Rate by State", 
x = "Deaths per Day per 100,000", 
y = "Vaccination Rate", color = "state") + 
theme_minimal()